<a href="https://colab.research.google.com/github/Vdmtx/nexus-eter/blob/main/extra%C3%A7a%C3%B5_de_legendas_de_filmes_08_07_2025_20_26_hs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🎬 Gerador de Legendas v3 (Seleção Individual)
#@markdown ### **Passo 1: Execute o Programa**
#@markdown Clique no **botão de Play (▶️)** à esquerda para iniciar.
#@markdown ### **Passo 2: Envie Seus Arquivos**
#@markdown Um botão para "Escolher arquivos" aparecerá. Envie **TODOS** os arquivos que deseja processar.
#@markdown ### **Passo 3: Configure Cada Arquivo**
#@markdown O programa irá mostrar um menu de idiomas **para cada arquivo que você enviou**. Siga as instruções.

# --- INSTALAÇÃO E CARREGAMENTO (Automático) ---
#@markdown ---
#@markdown **Não é necessário alterar o código abaixo.**
print("⏳ Instalando dependências...")
!pip install -q faster-whisper transformers srt tqdm

import os
import zipfile
from tqdm import tqdm
from faster_whisper import WhisperModel
from datetime import timedelta
import srt
from transformers import pipeline
import torch
from google.colab import files

# Dicionário de idiomas
IDIOMAS = {
    # Ordenado alfabeticamente para consistência do menu
    "Alemão": "de", "Cebuano": "ceb", "Chinês": "zh", "Coreano": "ko",
    "Espanhol": "es", "Francês": "fr", "Inglês": "en", "Italiano": "it",
    "Japonês": "ja", "Português": "pt", "Russo": "ru", "Tagalog": "tl"
}

def selecionar_idiomas_para_arquivo(idiomas_dict, nome_arquivo):
    """Exibe um menu numerado para um arquivo específico."""
    opcoes = list(idiomas_dict.keys())

    print("\n" + "="*50)
    print(f"  CONFIGURANDO O ARQUIVO: {nome_arquivo}")
    print("="*50)
    print("    SELEÇÃO DE IDIOMAS")
    print("="*50)
    for i, nome in enumerate(opcoes):
        print(f"  {i+1:<2} - {nome}")
    print("="*50)

    mapa_indices = {nome: i+1 for i, nome in enumerate(opcoes)}
    default_origem = mapa_indices.get("Inglês", 1)
    default_destino = mapa_indices.get("Português", 1)

    # Obter idioma de ORIGEM
    while True:
        try:
            prompt = f"🗣️ Digite o NÚMERO do idioma original para este arquivo [Padrão: {default_origem}]: "
            num_origem = int(input(prompt) or default_origem)
            if 1 <= num_origem <= len(opcoes):
                lang_origem_nome = opcoes[num_origem - 1]
                break
            else:
                print(f"❌ Erro: Por favor, digite um número entre 1 e {len(opcoes)}.")
        except ValueError:
            print("❌ Erro: Entrada inválida. Por favor, digite apenas o número.")

    # Obter idioma de DESTINO
    while True:
        try:
            prompt = f"🌐 Digite o NÚMERO da legenda para este arquivo [Padrão: {default_destino}]: "
            num_destino = int(input(prompt) or default_destino)
            if 1 <= num_destino <= len(opcoes):
                lang_destino_nome = opcoes[num_destino - 1]
                break
            else:
                print(f"❌ Erro: Por favor, digite um número entre 1 e {len(opcoes)}.")
        except ValueError:
            print("❌ Erro: Entrada inválida. Por favor, digite apenas o número.")

    lang_origem_code = idiomas_dict[lang_origem_nome]
    lang_destino_code = idiomas_dict[lang_destino_nome]

    print(f"👍 Configuração para '{nome_arquivo}': de [{lang_origem_nome}] para [{lang_destino_nome}].")
    return lang_origem_code, lang_destino_code

def setup_environment():
    """Prepara os diretórios e carrega os modelos."""
    print("\n🧠 Carregando modelos de Inteligência Artificial... Isso pode demorar alguns minutos.")
    global model_whisper, model_translator

    os.makedirs("videos_upload", exist_ok=True)
    os.makedirs("legendas_output", exist_ok=True)

    try:
        device = "cuda" if torch.cuda.is_available() else "cpu"
        compute_type = "float16" if torch.cuda.is_available() else "int8"
        model_whisper = WhisperModel("large-v2", device=device, compute_type=compute_type)
        model_translator = pipeline("translation", model="facebook/m2m100_418M", device=0 if device=="cuda" else -1)
        print(f"✅ Modelos carregados com sucesso! Usando: {device.upper()}")
        return True
    except Exception as e:
        print(f"❌ Erro ao carregar os modelos: {e}")
        return False

# --- EXECUÇÃO PRINCIPAL ---

if setup_environment():
    # 1. FAZER UPLOAD DE TODOS OS ARQUIVOS PRIMEIRO
    print("\n" + "="*50)
    print("📤 Por favor, faça o upload de TODOS os seus arquivos de uma vez.")
    uploaded = files.upload()

    if not uploaded:
        print("\nNenhum arquivo enviado. O programa foi encerrado.")
    else:
        # 2. PROCESSAR CADA ARQUIVO INDIVIDUALMENTE
        for filename, content in tqdm(uploaded.items(), desc="Progresso Geral"):
            # A SELEÇÃO DE IDIOMA AGORA É FEITA AQUI, DENTRO DO LOOP
            lang_origem_code, lang_destino_code = selecionar_idiomas_para_arquivo(IDIOMAS, filename)

            print(f"\nProcessando: {filename}...")
            filepath = os.path.join("videos_upload", filename)
            with open(filepath, 'wb') as f:
                f.write(content)

            segmentos, _ = model_whisper.transcribe(filepath, language=lang_origem_code, beam_size=5)

            segmentos_lista = list(segmentos)
            textos_originais = [seg.text.strip() for seg in segmentos_lista]

            if lang_origem_code != lang_destino_code:
                print("Traduzindo texto...")
                textos_finais = model_translator(textos_originais, src_lang=lang_origem_code, tgt_lang=lang_destino_code)
                textos_finais = [t['translation_text'] for t in textos_finais]
            else:
                textos_finais = textos_originais

            nome_base = os.path.splitext(filename)[0]
            nome_srt = f"{nome_base}_{lang_destino_code}.srt"
            caminho_srt = os.path.join("legendas_output", nome_srt)

            legendas = []
            for i, seg in enumerate(segmentos_lista):
                legenda = srt.Subtitle(
                    index=i + 1,
                    start=timedelta(seconds=seg.start),
                    end=timedelta(seconds=seg.end),
                    content=textos_finais[i]
                )
                legendas.append(legenda)

            with open(caminho_srt, "w", encoding="utf-8") as f:
                f.write(srt.compose(legendas))
            print(f"Legenda salva: {nome_srt}")

        # 3. COMPACTAR E BAIXAR NO FINAL DE TUDO
        print("\n" + "="*50)
        print("🗜️ Compactando todas as legendas em um arquivo .zip...")
        zip_path = "legendas_geradas.zip"
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for file in os.listdir("legendas_output"):
                zipf.write(os.path.join("legendas_output", file), arcname=file)

        print(f"✅ Processo concluído! Iniciando o download de '{zip_path}'.")
        files.download(zip_path)

⏳ Instalando dependências...

🧠 Carregando modelos de Inteligência Artificial... Isso pode demorar alguns minutos.


Device set to use cuda:0


✅ Modelos carregados com sucesso! Usando: CUDA

📤 Por favor, faça o upload de TODOS os seus arquivos de uma vez.
